In [3]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# 1. 数据加载与初步清洗

def load_and_clean_data(file_path):
    # 读取Excel数据
    df = pd.read_excel(file_path, engine = 'openpyxl')
    
    # 删除无关列
    drop_cols = ['安装日期', '工程单', '工程总数', '工程单号', '内机编号', '信息编号', '故障原因描述', '要求服务方式', '要求服务类型', '购买商场', '多次维修', '反映问题描述', '实务类际服型']
    df = df.drop(columns=[col for col in drop_cols if col in df.columns])
    
    # 基础清洗
    df = df.drop_duplicates()  # 删除重复记录
    
    # 日期字段转换
    date_cols = ['购机日期', '预约日期', '受理时间', '派工时间']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    if '购机日期' in df.columns and '预约日期' in df.columns:
        # 计算设备年龄（单位：月）
        df['设备年龄(月)'] = ((df['预约日期'] - df['购机日期']).dt.days / 30).round(1)
        # 处理异常值
        # - 年龄小于0：设为0
        # - 年龄大于20年：设为20年（240个月）
        df.loc[df['设备年龄(月)'] < 0, '设备年龄(月)'] = 0
        df.loc[df['设备年龄(月)'] > 240, '设备年龄(月)'] = 240
    else:
        print("警告：缺少购机日期或预约日期，无法计算设备年龄")
        df['设备年龄(月)'] = np.nan
        
    if '受理时间' in df.columns and '派工时间' in df.columns:    
        # 计算响应时间（分钟）
        df['响应时间(分钟)'] = ((df['派工时间'] - df['受理时间']).dt.total_seconds() / 60).round(1)
        # 处理负响应时间（设为0）
        df.loc[df['响应时间(分钟)'] < 0, '响应时间(分钟)'] = 0
        
        # ===== 添加响应时间分类 =====
        bins = [0, 30, 60, 120, float('inf')]
        labels = ['立即响应(<30m)', '快速响应(30-60m)', '一般响应(1-2h)', '延迟响应(>2h)']
        df['响应等级'] = pd.cut(
            df['响应时间(分钟)'], 
            bins=bins, 
            labels=labels,
            include_lowest=True
        )
    else:
        print("警告：缺少受理时间或派工时间，无法计算响应时间")
        df['响应时间(分钟)'] = np.nan
        df['响应等级'] = np.nan
    
    return df


# 2. 服务商代码处理
def process_service_code(df):
    if '服务商代码' not in df.columns:
        return df
    
    # 清洗服务商代码
    def clean_service_code(code):
        code = str(code).strip().upper()
        code = re.sub(r'[^A-Z0-9-]', '', code)  # 移除非字母数字字符
        
        # 补全缺失分隔符
        if not re.match(r'^[A-Z]-[A-Z]{2}-\d{3}-[A-Z0-9]+$', code):
            code = re.sub(r'([A-Z])(\d{3})', r'\1-\2', code)
        
        # 分段验证
        parts = code.split('-')
        if len(parts) < 4:
            code = '-'.join(parts + ['0000']*(4-len(parts)))
        
        return code
    
    df['服务商代码_清洗'] = df['服务商代码'].apply(clean_service_code)
    
    # 区域编码映射
    region_map = {
        '010': '北京', '020': '广州', '021': '上海', '022': '天津', '028': '成都', '395': '漯河', '531': '济南', 
        '023': '重庆', '024': '沈阳', '025': '南京', '027': '武汉', '029': '西安', '451': '哈尔滨', '532': '青岛',
        '591': '福州', '755': '深圳', '791': '南昌', '471': '呼和浩特', '411': '大连', '510': '无锡', '533': '淄博',
        '577': '温州', '931': '兰州', '311': '石家庄', '351': '太原', '371': '郑州', '516': '徐州', '535': '烟台',
        '536': '潍坊', '537': '济宁', '539': '临沂', '551': '合肥', '571': '杭州', '710': '襄阳', '731': '长沙',
        '771': '南宁', '830': '泸州', '851': '贵阳', '871': '昆明', '898': '海南', '991': '乌鲁木齐' 
    }
    # 城市到省份的映射字典
    city_to_province = {
    # 直辖市
    '北京': '北京', '上海': '上海', '天津': '天津', '重庆': '重庆',
    # 广东省
    '广州': '广东', '深圳': '广东',
    # 四川省
    '成都': '四川', '泸州': '四川',
    # 河南省
    '漯河': '河南', '郑州': '河南',
    # 山东省
    '济南': '山东', '青岛': '山东', '淄博': '山东', '烟台': '山东', '潍坊': '山东', '济宁': '山东', '临沂': '山东',
    # 辽宁省
    '沈阳': '辽宁', '大连': '辽宁',
    # 江苏省
    '南京': '江苏', '无锡': '江苏', '徐州': '江苏',
    # 湖北省
    '武汉': '湖北', '襄阳': '湖北',
    # 陕西省
    '西安': '陕西',
    # 黑龙江省
    '哈尔滨': '黑龙江',
    # 福建省
    '福州': '福建',
    # 江西省
    '南昌': '江西',
    # 内蒙古
    '呼和浩特': '内蒙古',
    # 浙江省
    '温州': '浙江', '杭州': '浙江',
    # 甘肃省
    '兰州': '甘肃',
    # 河北省
    '石家庄': '河北',
    # 山西省
    '太原': '山西',
    # 安徽省
    '合肥': '安徽',
    # 湖南省
    '长沙': '湖南',
    # 广西
    '南宁': '广西',
    # 贵州省
    '贵阳': '贵州',
    # 云南省
    '昆明': '云南',
    # 海南省
    '海南': '海南',
    # 新疆
    '乌鲁木齐': '新疆'
    }

    # 省份到区域的映射
    province_to_region = {
    '北京': '华北', '天津': '华北', '河北': '华北', '山西': '华北', '内蒙古': '华北',
    '辽宁': '东北', '吉林': '东北', '黑龙江': '东北',
    '上海': '华东', '江苏': '华东', '浙江': '华东', '安徽': '华东', '福建': '华东', '江西': '华东', '山东': '华东',
    '河南': '华中', '湖北': '华中', '湖南': '华中',
    '广东': '华南', '广西': '华南', '海南': '华南',
    '重庆': '西南', '四川': '西南', '贵州': '西南', '云南': '西南', '西藏': '西南',
    '陕西': '西北', '甘肃': '西北', '青海': '西北', '宁夏': '西北', '新疆': '西北'
    }
    
    # 特征提取
    df['服务商类型'] = df['服务商代码_清洗'].str[0]  # K/P
    df['区域编码'] = df['服务商代码_清洗'].str.extract(r'-(\d{3})-')[0]
    df['服务区域'] = df['区域编码'].map(region_map).fillna('其他')
    
    # 序列号解析
    df['序列前缀'] = df['服务商代码_清洗'].str[-4:].str.extract(r'([A-Z]+)')[0]
    df['序列编号'] = df['服务商代码_清洗'].str[-4:].str.extract(r'(\d+)')[0].astype(float)
    
    # 创建服务等级特征
    df['服务等级'] = np.where(
        df['序列前缀'].notnull(),
        df['序列前缀'],
        np.where(df['序列编号'] < 1000, 'A', 'B')
    )
    
    return df

def map_region_details(code):
    """
    根据区域编码获取完整的区域信息
    返回格式: (城市, 省份, 大区)
    """
    # 获取城市名称
    city = region_map.get(str(code), "未知")
    
    # 获取省份
    province = city_to_province.get(city, "未知")
    
    # 处理直辖市特殊情况
    if city in ['北京', '上海', '天津', '重庆']:
        province = city
    
    # 获取大区
    region = province_to_region.get(province, "其他")
    
    # 处理海南特殊情况
    if city == "海南":
        province = "海南"
        region = "华南"
    
    return city, province, region

def enhanced_region_mapping(df):
    if '区域编码' not in df.columns:
        # 直接从服务商代码提取
        df['区域编码'] = df['服务商代码_清洗'].str.extract(r'-(\d{3})-')[0].fillna('000')
    
    # 应用映射函数
    region_data = df['区域编码'].apply(map_region_details).apply(pd.Series)
    region_data.columns = ['服务城市', '服务省份', '服务大区']
    
    # 合并到数据框
    df = pd.concat([df, region_data], axis=1)
    
    # 添加城市等级特征
    tier1_cities = ['北京', '上海', '广州', '深圳']
    df['城市等级'] = df['服务城市'].apply(
        lambda x: '一线' if x in tier1_cities else 
                 '省会' if x in city_to_province.values() and x not in tier1_cities else 
                 '其他'
    )
    
    return df
    
# 3. 序列号解析

def process_serial_number(df):
    if '序列号' not in df.columns:
        return df
    
    # 序列号解码函数
    def decode_serial(serial):
        serial = str(serial).strip()
        return {
            "产线代码": serial[0:2] if len(serial) >= 2 else "NA",
            "日期批次": serial[2:8] if len(serial) >= 8 else "000000",
            "供应商代码": serial[14:17] if len(serial) >= 17 else "NA",
            "序列标识": serial[17:22] if len(serial) >= 22 else "NA"
        }
    
    # 应用解码
    df["序列解析"] = df["序列号"].apply(decode_serial)
    df = pd.concat([df, df["序列解析"].apply(pd.Series)], axis=1)
    
    # 修复日期批次
    def fix_date_batch(s):
        if isinstance(s, str) and s.isdigit() and len(s) == 6:
            return s
        return "010100"  # 设为默认值
    
    df['日期批次'] = df['日期批次'].apply(fix_date_batch)
    
    # 生产日期计算
    df['生产日期'] = pd.to_datetime(
        df['日期批次'].str[:2] + df['日期批次'].str[2:4] + '20' + df['日期批次'].str[4:6],
        format='%m%d%Y',
        errors='coerce'
    )
    
    # 质检等级提取
    df['质检等级'] = df['序列标识'].str[0]

    # 质检等级提取
    df['质检等级'] = df['序列标识'].str[0]
    
    # 验证生产日期与购机日期的逻辑关系
    if '购机日期' in df.columns and '生产日期' in df.columns:
        # 生产日期不应晚于购机日期
        invalid_mask = df['生产日期'] > df['购机日期']
        if invalid_mask.any():
            print(f"发现 {invalid_mask.sum()} 条生产日期晚于购机日期的异常记录")
            # 修正：将异常生产日期设为购机日期前30天
            df.loc[invalid_mask, '生产日期'] = df.loc[invalid_mask, '购机日期'] - pd.Timedelta(days=30)
    
    print(f"序列号解析完成，新增{len(df['序列解析'].apply(pd.Series).columns)}个特征")
    
    return df

# ======================
# 4. 保修类型处理
# ======================
def process_warranty(df):
    if '保修类型' not in df.columns:
        return df
    
    # 标准化保修类型
    warranty_map = {
        '在保': '保修期内', '过保': '已过保修', 'ext-warr': '延保',
        '保内': '保修期内', 'None': '未知', 'nan': '未知'
    }
    df['保修类型'] = df['保修类型'].map(warranty_map).fillna('未知')
    
    # 创建布尔特征
    df['是否在保'] = df['保修类型'].apply(lambda x: 1 if x == '保修期内' else 0)
    df['是否过保'] = df['保修类型'].apply(lambda x: 1 if x == '已过保修' else 0)
    
    # 基于设备年龄推断未知保修类型
    if '设备年龄(月)' in df.columns:
        df.loc[df['保修类型'] == '未知', '保修类型'] = df['设备年龄(月)'].apply(
            lambda x: '保修期内' if x < 12 else ('延保' if 12 <= x <= 36 else '已过保修')
        )
    
    return df

# ======================
# 5. 购买价格处理
# ======================
def process_purchase_price(df):
    if '购买价格' not in df.columns:
        return df

    # 确保产品型号是字符串类型且清理空格
    if '产品型号' in df.columns:
        df['产品型号'] = df['产品型号'].astype(str).str.strip()
        print(f"产品型号唯一值数量: {df['产品型号'].nunique()}")
    else:
        print("警告：缺少产品型号列")
        
    # 分离零价和非零价数据
    zero_price_mask = (df['购买价格'] == 0) | (df['购买价格'].isna())
    non_zero_df = df[~zero_price_mask].copy()
    zero_price_df = df[zero_price_mask].copy()

    # 创建价格映射字典
    model_price_map = {}
    market_price_map = {}
    
    # 只有存在非零记录时才创建映射
    if len(non_zero_df) > 0:
        # 1. 产品型号价格映射
        if '产品型号' in non_zero_df.columns:
            try:
                # 直接使用原始产品型号分组
                model_price_map = non_zero_df.groupby('产品型号')['购买价格'].median().to_dict()
                print(f"创建产品型号价格映射: {len(model_price_map)} 个型号")
            except Exception as e:
                print(f"产品型号分组错误: {e}")
        
        # 2. 市场级别价格映射
        if '市场级别' in non_zero_df.columns:
            try:
                non_zero_df['市场级别'] = non_zero_df['市场级别'].astype(str).str.strip()
                market_price_map = non_zero_df.groupby('市场级别')['购买价格'].median().to_dict()
                print(f"创建市场级别价格映射: {len(market_price_map)} 个级别")
            except Exception as e:
                print(f"市场级别分组错误: {e}")
        
        # 填充函数
    def fill_price(row):
        # 优先用型号价格
        if row.get('产品型号') and row['产品型号'] in model_price_map:
            return model_price_map[row['产品型号']]
        
        # 其次用市场级别价格
        if row.get('市场级别') and row['市场级别'] in market_price_map:
            return market_price_map[row['市场级别']]
        
        # 最后用全局中位数（如果有非零记录）
        if len(non_zero_df) > 0:
            return non_zero_df['购买价格'].median()
        
        return 0  # 默认值
        
        # 应用填充
    if len(zero_price_df) > 0:
        zero_price_df['购买价格_修正'] = zero_price_df.apply(fill_price, axis=1)
        zero_price_df['设备来源'] = '价格缺失'
        
        
        # 合并回原始数据
    if len(non_zero_df) > 0:
        non_zero_df['购买价格_修正'] = non_zero_df['购买价格']
        non_zero_df['设备来源'] = '正常购买'
        df = pd.concat([non_zero_df, zero_price_df], axis=0)
    else:
        df['购买价格_修正'] = df['购买价格']
        df['设备来源'] = '价格全缺失'
    
    # 创建价格等级
    if '购买价格_修正' in df.columns and df['购买价格_修正'].notna().any():
        try:
            # 获取有效价格数据
            valid_prices = df['购买价格_修正'][df['购买价格_修正'] > 0]
            
            if len(valid_prices) >= 5:
                # 使用实际分位数
                quantiles = [0, 0.2, 0.4, 0.6, 0.8, 1]
                labels = ['最低', '低', '中', '高', '最高']
                df['价格等级'] = pd.qcut(
                    df['购买价格_修正'], 
                    q=quantiles,
                    labels=labels,
                    duplicates='drop'
                )
            else:
                # 分为三档
                mean_price = valid_prices.mean()
                df['价格等级'] = pd.cut(
                    df['购买价格_修正'],
                    bins=[-1, mean_price*0.7, mean_price*1.3, float('inf')],
                    labels=['低', '中', '高'],
                    include_lowest=True
                )
        except Exception as e:
            print(f"创建价格等级错误: {e}")
            df['价格等级'] = '中'
    else:
        df['价格等级'] = '未知'
    
    print(f"修正后零价记录数: {len(df[df['购买价格_修正'] == 0])}")
    print("购买价格处理完成!")
    print("="*50)
    return df

# ======================
# 6. 特征工程管道
# ======================
def create_feature_pipeline():
    # 数值型特征处理
    numeric_features = ['设备年龄(月)']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # 分类特征处理
    categorical_features = ['市场级别', '服务区域', '服务等级']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    
    # 组合转换器
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features),
        ])
    
    return preprocessor

# ======================
# 主执行函数
# ======================
def main_processing(file_path):
    # 1. 加载和基础清洗
    print("="*50)
    print("步骤1: 数据加载与基础清洗")
    df = load_and_clean_data(file_path)
    
    # 2. 服务商代码处理
    print("\n步骤2: 服务商代码处理")
    df = process_service_code(df)
    
    # 3. 序列号解析
    print("\n步骤3: 序列号解析")
    df = process_serial_number(df)
    
    # 4. 保修类型处理
    print("\n步骤4: 保修类型处理")
    df = process_warranty(df)
    
    # 5. 购买价格处理
    df = process_purchase_price(df)
    
    # 6. 创建特征工程管道
    print("\n步骤6: 创建特征工程管道")
    preprocessor = create_feature_pipeline()
    
    # 7. 应用特征工程
    print("应用特征工程...")
    feature_data = preprocessor.fit_transform(df)
    
    # 8. 保存处理结果
    output_path = file_path.replace('.xlsx', '_processed.csv')
    df.to_csv(output_path, index=False)
    
    print(f"数据处理完成! 结果保存至: {output_path}")
    print(f"原始列数: {len(pd.read_excel(file_path).columns)} → 处理后列数: {len(df.columns)}")
    
    return df, feature_data

# 执行处理
if __name__ == "__main__":
    processed_df, features = main_processing(r"C:\Personal\Desktop\data2.xlsx")
    
    # 输出关键统计
    print("\n关键统计信息:")
    print(f"- 服务商类型分布:\n{processed_df['服务商类型'].value_counts()}")
    print(f"- 服务区域分布:\n{processed_df['服务区域'].value_counts()}")
    print(f"- 保修类型分布:\n{processed_df['保修类型'].value_counts()}")
    print(f"- 设备来源分布:\n{processed_df['设备来源'].value_counts()}")
    print(f"- 价格修正情况: 零价设备比例从 {len(processed_df[processed_df['购买价格']==0])/len(processed_df):.1%} "
          f"降至 {len(processed_df[processed_df['购买价格_修正']==0])/len(processed_df):.1%}")

步骤1: 数据加载与基础清洗

步骤2: 服务商代码处理

步骤3: 序列号解析
发现 22 条生产日期晚于购机日期的异常记录
序列号解析完成，新增4个特征

步骤4: 保修类型处理
产品型号唯一值数量: 3062
创建产品型号价格映射: 334 个型号
创建市场级别价格映射: 4 个级别
修正后零价记录数: 0
购买价格处理完成!

步骤6: 创建特征工程管道
应用特征工程...
数据处理完成! 结果保存至: C:\Personal\Desktop\data2_processed.csv
原始列数: 29 → 处理后列数: 38

关键统计信息:
- 服务商类型分布:
服务商类型
K    73050
P    10331
N      369
H        4
Name: count, dtype: int64
- 服务区域分布:
服务区域
北京      4488
重庆      4276
长沙      3977
广州      3534
福州      3502
成都      3400
深圳      3200
武汉      3050
南昌      3035
青岛      2691
南宁      2556
太原      2392
天津      2354
西安      2343
郑州      2330
杭州      2258
南京      2184
上海      2120
合肥      1984
临沂      1910
济南      1896
石家庄     1887
温州      1676
大连      1634
漯河      1537
沈阳      1518
无锡      1499
徐州      1419
济宁      1344
海南      1259
其他      1203
贵阳      1113
淄博      1072
乌鲁木齐    1026
哈尔滨     1020
潍坊       970
呼和浩特     966
兰州       882
昆明       782
烟台       767
襄阳       693
泸州         7
Name: count, dtype: int64
- 保修类型分布:
保修类型
保修期内    57230
已过保修    20415
延保   